# SWAST Forecasting Tool

An ensemble of Regression with ARIMA Errors and Facebook Prophet

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

check statsmodels version

In [2]:
import statsmodels as sm
print(sm.__version__)

import os

os.getcwd()

0.11.1


'/home/tom/Documents/code/swast-forecast-tool'

In [3]:
from swast_forecast.utility import (pre_process_daily_data, 
                                    default_ensemble,
                                    forecast, 
                                    multi_region_forecast)

Importing plotly failed. Interactive plots will not work.


## Constants

In [4]:
PATH = '../ambo_data/Daily_Responses_5_Years_2019_full.csv'

## Read in the data

In [5]:
clean = pre_process_daily_data(path=PATH, 
                               observation_col='Actual_Value', 
                               index_col='Actual_dt')
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Creating and fitting an Ensemble model to a region

The easy way to create an ensemble model is to call the `default_ensemble()` function form the utility module.  This returns the best known forecasting model.

In [6]:
model = default_ensemble()
model

ProphetARIMAEnsemble(order=(1, 1, 3), seasonal_order=(1, 0, 1, 7), prophet_default_alpha={self.alpha})

The code above informs us that the ensemble includes a Regression model with ARIMA errors with parameters (1, 1, 3)(1, 0, 1, 7).  By default a Prophet model will create a 80\% prediction interval 100(1-alpha)

To fit we call the `.fit()` method and pass in a `pd.Series` (or `pd.DataFrame`) that contains the historical observations.  By default you do not need to pass in holidays.  The ensemble will model new years day automatically (via Prophet's holidays function and as a dummy variable in the Regression with ARIMA errors).

In [7]:
#example - fitting Wiltshire - this will take a few seconds.
model.fit(clean['Wiltshire'])

## Forecasting an individual region.

Use the `.predict()` method to make a forecast.  The method takes 3 parameters:

* **horizon**: int - the forecast horizon e.g. 84 days
* **alpha**: float, optional (default=0.05) - a value between 0 and 1 and used to construct a 100(1 - alpha) prediction interval. E.g. alpha=0.2 returns a 80\% interval.  
* **return_all_models**: bool, optional (default=False). If sets to true returns the ensemble prediction AND the Prophet and Regression predictions.

In [10]:
#example 1: predict 7 days ahead - remember we have fitted Wiltshire training data.
forecast_frame = model.predict(horizon=7)
forecast_frame

,yhat,yhat_lower_95,yhat_upper95
ds,,,
2020-01-01,389.033671,352.591641,425.461895
2020-01-02,328.488021,291.628199,365.847111
2020-01-03,332.563272,295.599838,369.578768
2020-01-04,348.733984,312.142733,386.007147
2020-01-05,348.858335,311.818218,384.861011
2020-01-06,335.638559,297.267796,373.163166
2020-01-07,326.682607,288.813970,363.312506


The method returns a `pd.DataFrame` containing mean forecast (yhat) and an upper and lower prediction interval.  The code below demonstrates how to return predictions from both the ARIMA and Prophet models.  We will also return a different prediction interval.

In [11]:
#example 2: predict 7 days ahead, return 80% PI and individual model preds
forecast_frame = model.predict(horizon=7, alpha=0.2, return_all_models=True)
forecast_frame

,yhat,yhat_lower_80,yhat_upper80,arima_mean,arima_lower_80,arima_upper_80,prophet_mean,prophet_lower_80,prophet_upper_80
ds,,,,,,,,,
2020-01-01,389.033671,365.216675,412.021036,381.652918,357.811543,405.494293,396.414423,372.621808,418.547779
2020-01-02,328.488021,304.584836,352.553672,328.893489,304.534189,353.252789,328.082554,304.635482,351.854556
2020-01-03,332.563272,308.549435,356.265552,333.117808,308.470773,357.764844,332.008735,308.628097,354.766261
2020-01-04,348.733984,324.550447,372.660558,347.836168,322.927592,372.744744,349.631799,326.173302,372.576373
2020-01-05,348.858335,325.927962,374.416310,348.568032,323.420965,373.715099,349.148638,328.434960,375.117522
2020-01-06,335.638559,311.197656,360.184345,337.437881,312.072670,362.803092,333.839236,310.322642,357.565599
2020-01-07,326.682607,302.086976,350.943181,330.057359,304.492022,355.622696,323.307855,299.681929,346.263666


## An 'all in one' forecast function

As an alternative to the above the `utility` module contains a convenience function called `forecast`.  This is an all-in-one function.  Just pass in your training data (for a single time series) and horizon.

In [12]:
forecast(clean['Wiltshire'], 
         horizon=6, 
         alpha=0.2,
         return_all_models=False)

,yhat,yhat_lower_80,yhat_upper80
ds,,,
2020-01-01,389.033671,365.418638,411.813435
2020-01-02,328.488021,304.447560,353.369776
2020-01-03,332.563272,308.652606,357.045318
2020-01-04,348.733984,325.192503,372.853248
2020-01-05,348.858335,324.854773,373.090575
2020-01-06,335.638559,311.689260,360.813569


## Forecasting multiple regions in one go.

If there are multiple regions to forecast put all of the training data into the same frame (see `clean`) and pass this to the `multi_region_forecast()` function from the `utility` module.

This is an efficient function as it runs the forecasts in parrallel across your CPU cores.  E.g. if you have a 4 cores then 4 regions will be forecast simultaneously.  This will reduce model run time (assuming you have more than one Core).

In [13]:
#note depending on your machine this will take 20 seconds to run.
forecasts = multi_region_forecast(y_train=clean, horizon=7)

In [14]:
#the function returns a list of pd.DataFrame's
type(forecasts)

list

In [15]:
#results for BNSSG
forecasts[0]

,yhat,yhat_lower_95,yhat_upper95
ds,,,
2020-01-01,667.046404,621.206196,715.990973
2020-01-02,545.976607,498.818996,595.299378
2020-01-03,553.843168,503.506693,603.142045
2020-01-04,588.031241,539.256041,636.383141
2020-01-05,579.761990,529.775283,629.330352
2020-01-06,554.881950,506.283011,604.049023
2020-01-07,537.738373,488.384776,588.521742
2020-01-08,534.353752,484.632867,582.344943


In [16]:
#results for Cornwall is at index 1 etc.
forecasts[1]

,yhat,yhat_lower_95,yhat_upper95
ds,,,
2020-01-01,317.561603,286.059778,349.883321
2020-01-02,252.695551,219.958918,286.241994
2020-01-03,255.669320,223.372014,287.314004
2020-01-04,271.179998,238.070727,303.273731
2020-01-05,274.010956,240.328324,306.819183
2020-01-06,261.538837,228.391769,295.286874
2020-01-07,250.725712,218.711811,284.210147
2020-01-08,248.816783,215.832096,281.931705
